In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
%matplotlib qt5

In [2]:
import sys

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *

# Analytic signal

In [29]:
def RD_realtocomp2( RD ):

    Nf  = RD.size

    # if Nf is a multiple of 2, fftshift(fftshift(X)) = X
    if ( ( Nf % 2 ) != 0 ):
        error( "Nf should be a multiple of 2" )

    h = np.zeros( Nf )
    h[ 0 ] = 1
    h[ int( Nf / 2) ] = 1
    h[ 1 : int(Nf / 2) ] = 2

    #RDs = np.fft.ifft( np.fft.fftshift( RD ) )
    RDs = np.fft.ifft( RD )

    RDh = RDs * h

    #RDc = np.fft.fftshift( np.fft.fft( RDh ) )
    RDc = np.fft.fft( RDh )

    return RDc[::2]

## Compare RD_realtocomp and RD_realtocomp2

In [30]:
teta = np.arange( -10*np.pi, 10*np.pi, np.pi/10 )
phi0 = 30 * 180 / np.pi
y = teta
as_y = RD_realtocomp( y )
as_y2 = RD_realtocomp2( y )
fft_y = np.fft.fft( y )

In [31]:
as_y.shape, as_y2.shape

((100,), (100,))

In [46]:
plt.figure()

plt.subplot(121)
plt.plot( np.real( np.fft.fft( as_y ) ), "D-", label="as_y  (RD_realtocomp)" )
plt.plot( np.real( np.fft.fft( as_y2 ) ), '.-', label="as2_y ( (RD_realtocomp2)" )
plt.plot( np.real( fft_y ), label="fft( y )" )
plt.grid()
plt.legend()

plt.subplot(122)
plt.plot( np.imag( np.fft.fft( as_y ) ), "D-", label="as_y  (RD_realtocomp)" )
plt.plot( np.imag( np.fft.fft( as_y2 ) ), '.-', label="as2_y ( (RD_realtocomp2)" )
plt.plot( np.imag( fft_y ), label="fft( y )" )
plt.grid()
plt.legend()

## Compute the analytical signal using the Hilbert transform

In [33]:
def as2( y ):
    return signal.hilbert( y ) / 2

In [34]:
def as3( y ):
    fft_y = np.fft.fftshift( np.fft.fft( y ) )
    nbPoints = y.shape[0]
    nbPoints2 = int( nbPoints / 2 )
    fft_as_y = np.zeros( (nbPoints,), dtype=complex )
    fft_as_y[0] = fft_y[0]
    fft_as_y[ 1 : nbPoints2 ] = 2 * fft_y[ 1 : nbPoints2 ]
    fft_as_y[0] = fft_y[0]
    fft_as_y[nbPoints2] = fft_y[nbPoints2]
    return np.fft.ifft( np.fft.ifftshift( fft_as_y ) ) / 2

In [41]:
def as4( y ):
    fft_y = np.fft.fft( y )
    nbPoints = y.shape[0]
    nbPoints2 = int( nbPoints / 2 )
    fft_as_y = np.zeros( (nbPoints,), dtype=complex )
    fft_as_y[0] = fft_y[0]
    fft_as_y[ 1 : nbPoints2 ] = 2 * fft_y[ 1 : nbPoints2 ]
    fft_as_y[0] = fft_y[0]
    fft_as_y[nbPoints2] = fft_y[nbPoints2]
    return np.fft.ifft( fft_as_y ) / 2

In [42]:
teta = np.arange( -10*np.pi, 10*np.pi, np.pi/10 )
phi0 = 30 * 180 / np.pi
y = np.cos(teta - phi0) + np.cos( 2 * teta - phi0)
y = teta
expy = np.exp(1j * (teta-phi0) ) / 2
as_y = RD_realtocomp2( y )
as2_y = as2( y )
as3_y = as3( y )[::2]*2
as4_y = as4( y )[::2]*2
fft_y = np.fft.fft( y )
fft_expy = np.fft.fft( expy )

In [47]:
plt.figure()

plt.subplot(121)
plt.plot( np.real( np.fft.fft( as3_y ) ), "D-", label="as3_y" )
plt.plot( np.real( np.fft.fft( as3_y ) ), "D-", label="as4_y" )
plt.plot( np.real( np.fft.fft( as2_y ) ), '.-', label="as2_y (hilbert)" )
plt.plot( np.real( np.fft.fft( as_y ) ), "x-", label="as_y (RD_realtocomp)", color="yellow" )
plt.plot( np.real( fft_y ), label="fft_y" )
plt.grid()
plt.legend()

plt.subplot(122)
plt.plot( np.imag( np.fft.fft( as3_y ) ), "D-" )
plt.plot( np.imag( np.fft.fft( as4_y ) ), "D-" )
plt.plot( np.imag( np.fft.fft( as2_y ) ), '.-' )
plt.plot( np.imag( np.fft.fft( as_y ) ), "x-", color="yellow" )
plt.plot( np.imag( fft_y ) )
plt.grid()